<a href="https://colab.research.google.com/github/JSunkel/Forecasting-Models/blob/master/Churn_Reason_Prophet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from fbprophet import Prophet

In [0]:
# Load the full dataset
fact_table = pd.read_csv('drive/My Drive/Churn/churn_fact_table.csv')

In [42]:
# Check the full dataset format (testing only)
fact_table.head()

,ds,reason_port_out,reason_involuntary,reason_not_needed,reason_other,reason_death,reason_prepaid,reason_false_churn
0,1/1/2018,3324,2172,982,548,75,49,10
1,1/2/2018,6133,2284,1154,917,552,288,126
2,1/3/2018,4410,1339,117,209,395,258,95
3,1/4/2018,5137,2951,1622,1012,610,233,93
4,1/5/2018,1354,1400,215,342,355,210,97


In [0]:
# Split out indivudal datasets for forecasting
reason_port_out = fact_table[['ds', 'reason_port_out']].rename(columns = {'reason_port_out': 'y'})
reason_involuntary = fact_table[['ds', 'reason_involuntary']].rename(columns = {'reason_involuntary': 'y'})
reason_not_needed = fact_table[['ds', 'reason_not_needed']].rename(columns = {'reason_not_needed': 'y'})
reason_other = fact_table[['ds', 'reason_other']].rename(columns = {'reason_other': 'y'})
reason_death = fact_table[['ds', 'reason_death']].rename(columns = {'reason_death': 'y'})
reason_prepaid = fact_table[['ds', 'reason_prepaid']].rename(columns = {'reason_prepaid': 'y'})
reason_false_churn = fact_table[['ds', 'reason_false_churn']].rename(columns = {'reason_false_churn': 'y'})

In [46]:
# Check the individual dataset format (testing only)
reason_port_out.head()

,ds,y
0,1/1/2018,3324
1,1/2/2018,6133
2,1/3/2018,4410
3,1/4/2018,5137
4,1/5/2018,1354


In [0]:
# Define the holidays
jan_bill_cycle = pd.DataFrame({
    'holiday': 'jan_bill_cycle',
    'ds': pd.to_datetime(['2018-01-05',
                          '2018-01-11',
                          '2018-01-14',
                          '2018-01-17',
                          '2018-01-24',
                          '2018-01-27',
                          '2019-01-29',
                          '2018-01-30',
                          '2018-01-31',
                          '2019-01-05',
                          '2019-01-11',
                          '2019-01-14',
                          '2019-01-17',
                          '2019-01-24',
                          '2019-01-27',
                          '2019-01-29',
                          '2019-01-30',
                          '2019-01-31',
                          '2020-01-05',
                          '2020-01-11',
                          '2020-01-14',
                          '2020-01-17',
                          '2020-01-24',
                          '2020-01-27',
                          '2020-01-29',
                          '2020-01-30',
                          '2020-01-31']),
    'lower_window': 0,
    'upper_window': 0,
})

feb_bill_cycle = pd.DataFrame({
    'holiday': 'feb_bill_cycle',
    'ds': pd.to_datetime(['2018-02-05',
                          '2018-02-11',
                          '2018-02-14',
                          '2018-02-17',
                          '2018-02-24',
                          '2018-02-27',
                          '2019-02-05',
                          '2019-02-11',
                          '2019-02-14',
                          '2019-02-17',
                          '2019-02-24',
                          '2019-02-27',
                          '2020-02-05',
                          '2020-02-11',
                          '2020-02-14',
                          '2020-02-17',
                          '2020-02-24',
                          '2020-02-27',
                          '2020-02-29']),
    'lower_window': 0,
    'upper_window': 0,
})

mar_bill_cycle = pd.DataFrame({
    'holiday': 'mar_bill_cycle',
    'ds': pd.to_datetime(['2018-03-05',
                          '2018-03-11',
                          '2018-03-14',
                          '2018-03-17',
                          '2018-03-24',
                          '2018-03-27',
                          '2019-03-29',
                          '2018-03-30',
                          '2018-03-31',
                          '2019-03-05',
                          '2019-03-11',
                          '2019-03-14',
                          '2019-03-17',
                          '2019-03-24',
                          '2019-03-27',
                          '2019-03-29',
                          '2019-03-30',
                          '2019-03-31',
                          '2020-03-05',
                          '2020-03-11',
                          '2020-03-14',
                          '2020-03-17',
                          '2020-03-24',
                          '2020-03-27',
                          '2020-03-29',
                          '2020-03-30',
                          '2020-03-31']),
    'lower_window': 0,
    'upper_window': 0,
})

apr_bill_cycle = pd.DataFrame({
    'holiday': 'apr_bill_cycle',
    'ds': pd.to_datetime(['2018-04-05',
                          '2018-04-11',
                          '2018-04-14',
                          '2018-04-17',
                          '2018-04-24',
                          '2018-04-27',
                          '2019-04-29',
                          '2018-04-30',
                          '2019-04-05',
                          '2019-04-11',
                          '2019-04-14',
                          '2019-04-17',
                          '2019-04-24',
                          '2019-04-27',
                          '2019-04-29',
                          '2019-04-30',
                          '2020-04-05',
                          '2020-04-11',
                          '2020-04-14',
                          '2020-04-17',
                          '2020-04-24',
                          '2020-04-27',
                          '2020-04-29',
                          '2020-04-30']),
    'lower_window': 0,
    'upper_window': 0,
})

may_bill_cycle = pd.DataFrame({
    'holiday': 'may_bill_cycle',
    'ds': pd.to_datetime(['2018-05-05',
                          '2018-05-11',
                          '2018-05-14',
                          '2018-05-17',
                          '2018-05-24',
                          '2018-05-27',
                          '2019-05-29',
                          '2018-05-30',
                          '2018-05-31',
                          '2019-05-05',
                          '2019-05-11',
                          '2019-05-14',
                          '2019-05-17',
                          '2019-05-24',
                          '2019-05-27',
                          '2019-05-29',
                          '2019-05-30',
                          '2019-05-31',
                          '2020-05-05',
                          '2020-05-11',
                          '2020-05-14',
                          '2020-05-17',
                          '2020-05-24',
                          '2020-05-27',
                          '2020-05-29',
                          '2020-05-30',
                          '2020-05-31']),
    'lower_window': 0,
    'upper_window': 0,
})

jun_bill_cycle = pd.DataFrame({
    'holiday': 'jun_bill_cycle',
    'ds': pd.to_datetime(['2018-06-05',
                          '2018-06-11',
                          '2018-06-14',
                          '2018-06-17',
                          '2018-06-24',
                          '2018-06-27',
                          '2019-06-29',
                          '2018-06-30',
                          '2019-06-05',
                          '2019-06-11',
                          '2019-06-14',
                          '2019-06-17',
                          '2019-06-24',
                          '2019-06-27',
                          '2019-06-29',
                          '2019-06-30',
                          '2020-06-05',
                          '2020-06-11',
                          '2020-06-14',
                          '2020-06-17',
                          '2020-06-24',
                          '2020-06-27',
                          '2020-06-29',
                          '2020-06-30']),
    'lower_window': 0,
    'upper_window': 0,
})

jul_bill_cycle = pd.DataFrame({
    'holiday': 'jul_bill_cycle',
    'ds': pd.to_datetime(['2018-07-05',
                          '2018-07-11',
                          '2018-07-14',
                          '2018-07-17',
                          '2018-07-24',
                          '2018-07-27',
                          '2019-07-29',
                          '2018-07-30',
                          '2018-07-31',
                          '2019-07-05',
                          '2019-07-11',
                          '2019-07-14',
                          '2019-07-17',
                          '2019-07-24',
                          '2019-07-27',
                          '2019-07-29',
                          '2019-07-30',
                          '2019-07-31',
                          '2020-07-05',
                          '2020-07-11',
                          '2020-07-14',
                          '2020-07-17',
                          '2020-07-24',
                          '2020-07-27',
                          '2020-07-29',
                          '2020-07-30',
                          '2020-07-31']),
    'lower_window': 0,
    'upper_window': 0,
})

aug_bill_cycle = pd.DataFrame({
    'holiday': 'aug_bill_cycle',
    'ds': pd.to_datetime(['2018-08-05',
                          '2018-08-11',
                          '2018-08-14',
                          '2018-08-17',
                          '2018-08-24',
                          '2018-08-27',
                          '2019-08-29',
                          '2018-08-30',
                          '2018-08-31',
                          '2019-08-05',
                          '2019-08-11',
                          '2019-08-14',
                          '2019-08-17',
                          '2019-08-24',
                          '2019-08-27',
                          '2019-08-29',
                          '2019-08-30',
                          '2019-08-31',
                          '2020-08-05',
                          '2020-08-11',
                          '2020-08-14',
                          '2020-08-17',
                          '2020-08-24',
                          '2020-08-27',
                          '2020-08-29',
                          '2020-08-30',
                          '2020-08-31']),
    'lower_window': 0,
    'upper_window': 0,
})

sep_bill_cycle = pd.DataFrame({
    'holiday': 'sep_bill_cycle',
    'ds': pd.to_datetime(['2018-09-05',
                          '2018-09-11',
                          '2018-09-14',
                          '2018-09-17',
                          '2018-09-24',
                          '2018-09-27',
                          '2019-09-29',
                          '2018-09-30',
                          '2019-09-05',
                          '2019-09-11',
                          '2019-09-14',
                          '2019-09-17',
                          '2019-09-24',
                          '2019-09-27',
                          '2019-09-29',
                          '2019-09-30',
                          '2020-09-05',
                          '2020-09-11',
                          '2020-09-14',
                          '2020-09-17',
                          '2020-09-24',
                          '2020-09-27',
                          '2020-09-29',
                          '2020-09-30']),
    'lower_window': 0,
    'upper_window': 0,
})

oct_bill_cycle = pd.DataFrame({
    'holiday': 'oct_bill_cycle',
    'ds': pd.to_datetime(['2018-10-05',
                          '2018-10-11',
                          '2018-10-14',
                          '2018-10-17',
                          '2018-10-24',
                          '2018-10-27',
                          '2019-10-29',
                          '2018-10-30',
                          '2018-10-31',
                          '2019-10-05',
                          '2019-10-11',
                          '2019-10-14',
                          '2019-10-17',
                          '2019-10-24',
                          '2019-10-27',
                          '2019-10-29',
                          '2019-10-30',
                          '2019-10-31',
                          '2020-10-05',
                          '2020-10-11',
                          '2020-10-14',
                          '2020-10-17',
                          '2020-10-24',
                          '2020-10-27',
                          '2020-10-29',
                          '2020-10-30',
                          '2020-10-31']),
    'lower_window': 0,
    'upper_window': 0,
})

nov_bill_cycle = pd.DataFrame({
    'holiday': 'nov_bill_cycle',
    'ds': pd.to_datetime(['2018-11-05',
                          '2018-11-11',
                          '2018-11-14',
                          '2018-11-17',
                          '2018-11-24',
                          '2018-11-27',
                          '2019-11-29',
                          '2018-11-30',
                          '2019-11-05',
                          '2019-11-11',
                          '2019-11-14',
                          '2019-11-17',
                          '2019-11-24',
                          '2019-11-27',
                          '2019-11-29',
                          '2019-11-30',
                          '2020-11-05',
                          '2020-11-11',
                          '2020-11-14',
                          '2020-11-17',
                          '2020-11-24',
                          '2020-11-27',
                          '2020-11-29',
                          '2020-11-30']),
    'lower_window': 0,
    'upper_window': 0,
})

dec_bill_cycle = pd.DataFrame({
    'holiday': 'dec_bill_cycle',
    'ds': pd.to_datetime(['2018-12-05',
                          '2018-12-11',
                          '2018-12-14',
                          '2018-12-17',
                          '2018-12-24',
                          '2018-12-27',
                          '2019-12-29',
                          '2018-12-30',
                          '2018-12-31',
                          '2019-12-05',
                          '2019-12-11',
                          '2019-12-14',
                          '2019-12-17',
                          '2019-12-24',
                          '2019-12-27',
                          '2019-12-29',
                          '2019-12-30',
                          '2019-12-31',
                          '2020-12-05',
                          '2020-12-11',
                          '2020-12-14',
                          '2020-12-17',
                          '2020-12-24',
                          '2020-12-27',
                          '2020-12-29',
                          '2020-12-30',
                          '2020-12-31']),
    'lower_window': 0,
    'upper_window': 0,
})

In [0]:
holidays = pd.concat((jan_bill_cycle,
                      feb_bill_cycle,
                      mar_bill_cycle,
                      apr_bill_cycle,
                      may_bill_cycle,
                      jun_bill_cycle,
                      jul_bill_cycle,
                      aug_bill_cycle,
                      sep_bill_cycle,
                      oct_bill_cycle,
                      nov_bill_cycle,
                      dec_bill_cycle))

In [50]:
# Define and fit the models
reason_port_out_model = Prophet(holidays = holidays).fit(reason_port_out)
reason_involuntary_model = Prophet(holidays = holidays).fit(reason_involuntary)
reason_not_needed_model = Prophet(holidays = holidays).fit(reason_not_needed)
reason_other_model = Prophet(holidays = holidays).fit(reason_other)
reason_death_model = Prophet(holidays = holidays).fit(reason_death)
reason_prepaid_model = Prophet(holidays = holidays).fit(reason_prepaid)
reason_false_churn_model = Prophet(holidays = holidays).fit(reason_false_churn)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [0]:
# NUmber of periods into the future to forecast
n = 31

In [0]:
# Create dataframes to hold predictions
reason_port_out_future = reason_port_out_model.make_future_dataframe(periods = n)
reason_involuntary_future = reason_involuntary_model.make_future_dataframe(periods = n)
reason_not_needed_future = reason_not_needed_model.make_future_dataframe(periods = n)
reason_other_future = reason_other_model.make_future_dataframe(periods = n)
reason_death_future = reason_death_model.make_future_dataframe(periods = n)
reason_prepaid_future = reason_prepaid_model.make_future_dataframe(periods = n)
reason_false_churn_future = reason_false_churn_model.make_future_dataframe(periods = n)

In [0]:
# Make predictions
reason_port_out_forecast = reason_port_out_model.predict(reason_port_out_future)
reason_involuntary_forecast = reason_involuntary_model.predict(reason_involuntary_future)
reason_not_needed_forecast = reason_not_needed_model.predict(reason_not_needed_future)
reason_other_forecast = reason_other_model.predict(reason_other_future)
reason_death_forecast = reason_death_model.predict(reason_death_future)
reason_prepaid_forecast = reason_prepaid_model.predict(reason_prepaid_future)
reason_false_churn_forecast = reason_false_churn_model.predict(reason_false_churn_future)

In [0]:
# Get the datestamp and prediction columns
reason_port_out_forecast_subset = reason_port_out_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Reason_Port_Outs'})
reason_involuntary_forecast_subset = reason_involuntary_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Reason_Involuntary'})
reason_not_needed_forecast_subset = reason_not_needed_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Reason_Not_Needed'})
reason_other_forecast_subset = reason_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Reason_Other'})
reason_death_forecast_subset = reason_death_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Reason_Death'})
reason_prepaid_forecast_subset = reason_prepaid_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Reason_Prepaid'})
reason_false_churn_forecast_subset = reason_false_churn_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Reason_False_Churn'})

In [58]:
# Check the format of the subsetted forecasts (testing only)
reason_port_out_forecast_subset.tail()

,ds,Reason_Port_Outs
816,2020-03-27,1892.288259
817,2020-03-28,4936.795523
818,2020-03-29,1507.532217
819,2020-03-30,2157.106222
820,2020-03-31,1458.629178


In [0]:
# Combine the forecasts into one table
churn_pred_table = reason_port_out_forecast_subset
churn_pred_table.insert(2, 'Reason_involuntary', reason_involuntary_forecast_subset['Reason_Involuntary'])
churn_pred_table.insert(3, 'Reason_Not_Needed', reason_not_needed_forecast_subset['Reason_Not_Needed'])
churn_pred_table.insert(4, 'Reason_Other', reason_other_forecast_subset['Reason_Other'])
churn_pred_table.insert(5, 'Reason_Death', reason_death_forecast_subset['Reason_Death'])
churn_pred_table.insert(6, 'Reason_Prepaid', reason_prepaid_forecast_subset['Reason_Prepaid'])
churn_pred_table.insert(7, 'Reason_False_Churn', reason_false_churn_forecast_subset['Reason_False_Churn'])

In [60]:
# Check the format of the final predictions table (testing only)
churn_pred_table.tail()

,ds,Reason_Port_Outs,Reason_involuntary,Reason_Not_Needed,Reason_Other,Reason_Death,Reason_Prepaid,Reason_False_Churn
816,2020-03-27,1892.288259,2748.852539,177.675028,133.190712,415.578616,306.779060,139.694754
817,2020-03-28,4936.795523,2482.530725,745.970486,338.412444,257.420788,237.078975,107.954125
818,2020-03-29,1507.532217,2613.506186,118.045715,41.189885,154.310167,142.247561,74.623040
819,2020-03-30,2157.106222,2063.634721,169.244137,151.683121,489.405059,302.691375,156.999016
820,2020-03-31,1458.629178,2489.922881,181.365019,163.376473,469.706839,291.872907,150.670180


In [0]:
# Save the predictions table to a csv file
churn_pred_table.to_csv('drive/My Drive/Churn/churn_pred_table.csv')